# Importando e tratando as bases de dados

><h1>Importação dos dados

>><h2>Chuva_2019

In [ ]:
import json
import requests
from io import StringIO

# carregando as credenciais
with open("/home/daniel/geohmount/code/credentials.json", "r") as credentials_file:
    credentials = json.load(credentials_file)
# parametros da request (github info)
owner = credentials["github"]["username"]
token = credentials["github"]["token"]
repo = credentials["github"]["repo"]
path = 'sazonalidade-chuva/chuva_2019.csv'

# sending the request
r = requests.get(f'https://api.github.com/repos/{owner}/{repo}/contents/{path}',
    headers={
        'accept': 'application/vnd.github.v3.raw',
        'authorization': f'token {token}'
            }
    )

#convert string to StringIO object
data = StringIO(r.text)

In [ ]:
import pandas as pd

df = pd.read_csv(data, sep=';')
df

>><h2>Sazonalidade

In [ ]:
# parameters for the request
owner = credentials["github"]["username"]
token = credentials["github"]["token"]
repo = credentials["github"]["repo"]
path = 'sazonalidade-chuva/sazonalidade.csv'

# sending the request
r = requests.get(f'https://api.github.com/repos/{owner}/{repo}/contents/{path}',
    headers={
        'accept': 'application/vnd.github.v3.raw',
        'authorization': f'token {token}'
            }
    )

#convert string to StringIO object
data = StringIO(r.text)

In [ ]:
import pandas as pd

df2 = pd.read_csv(data, sep=';')
df2.head()

In [ ]:
# invertendo o dataframe
df2 = df2.reindex(index=df2.index[::-1])
df2.head()

In [ ]:
# renomeando as séries históricas
df2.rename(columns={'Série Histórica (2007-2019) e DP': 'sh_SM', 'Série Histórica (2007-2019) e DP.1': 'sh_BM', 'Série Histórica (2007-2019) e DP.2': 'sh_SB'}, inplace=True)
df2.head()

In [ ]:
import base64

# set a local image as the background
img = '/home/daniel/geohmount/code/logos-png/GEOHMOUNT-Logo-Cinzas.png'
geohmount_logo = base64.b64encode(open(img, 'rb').read())

# Criando o 1º gráfico

In [ ]:
config = {'displayModeBar': True, 'toImageButtonOptions': {'height': None, 'width': None}}

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Bar(name='Bonfim',x=df.estacao[:3], y=df.chuva[:3], marker_color='rgb(0, 153, 216)', error_y=dict(type='data', array=df.sd[:3])
))
fig.add_trace(go.Bar(name='Soberbo',x=df.estacao[3:5], y=df.chuva[3:5], marker_color='rgb(0, 90, 74)', error_y=dict(type='data', array=df.sd[3:5])
))
fig.add_trace(go.Bar(name='Santa Marta',x=df.estacao[5:8], y=df.chuva[5:8], marker_color='rgb(86, 34, 18)', error_y=dict(type='data', array=df.sd[5:8])
))

fig.add_layout_image(
    dict(
        source=f'data:image/png;base64,{geohmount_logo.decode()}',
        xref="paper",
        yref="paper",
        x=0.18,
        y=0.9,
        sizex=0.65,
        sizey=0.65,
        sizing="contain",
        opacity=0.13,
        layer="below")
)

fig.update_layout(template='plotly_white', width=1200, height=800, font_family='Open Sans',  xaxis_tickfont_size=16, yaxis=dict(title='<b>Precipitação no ano de 2019 (mm)', titlefont_size=16, tickfont_size=16), legend=dict(font_size=14))
fig.show()
#fig.write_html('C:\\Users\\daniel\\Documents\\Github\\geohmount\\geohmount-plotly\\sazonalidade-chuva\\chuva_2019.html', include_plotlyjs='cdn', config=config)

# Criando o 2º gráfico

In [ ]:
import plotly.graph_objects as go

blue = 'rgb(0, 153, 216)'
green = 'rgb(0, 90, 74)'
brown = 'rgb(86, 34, 18)'
fig2 = go.Figure()

fig2.add_trace(go.Bar(name='BM (Estação Pedro do Rio)', x=df2['mês'], y=df2['BM (Estação Pedro do Rio)'], marker=dict(color=blue)
))

fig2.add_trace(go.Scatter(mode='lines+markers', name='Série Histórica (2007-2019)', x=df2['mês'], y=df2['sh_BM'], error_y=dict(color='black', type='data', array=df2['sd_BM'] ), marker=dict(color=blue, line_width=1, line_color='black'), line=dict(color=blue, width=3), visible='legendonly'
))

fig2.add_trace(go.Bar(name='SB (Estação PARNASO)',x=df2['mês'], y=df2['SB (Estação PARNASO)'], marker=dict(color=green)
))

fig2.add_trace(go.Scatter(mode='lines+markers', name='Série Histórica (2007-2019)', x=df2['mês'], y=df2['sh_SB'], error_y=dict(color='black', type='data', array=df2['sd_SB']), marker=dict(color=green, line_width=1, line_color='black'), line=dict(color=green, width=3), visible='legendonly'
))

fig2.add_trace(go.Bar(name='SM (Estação Caparaó)',x=df2['mês'], y=df2['SM (Estação Caparaó)'], marker=dict(color=brown)
))

fig2.add_trace(go.Scatter(mode='lines+markers', name='Série Histórica (2007-2019)', x=df2['mês'], y=df2['sh_SM'], error_y=dict(color='black', type='data', array=df2['sd_SM']), marker=dict(color=brown, line_width=1, line_color='black'), line=dict(color=brown, width=3), visible='legendonly'
))

fig2.add_layout_image(
    dict(
        source=f'data:image/png;base64,{geohmount_logo.decode()}',
        xref="paper",
        yref="paper",
        x=0.23,
        y=0.9,
        sizex=0.5,
        sizey=0.5,
        sizing="contain",
        opacity=0.13,
        layer="below")
)

fig2.update_layout(template='plotly_white',
width=1500, height=800, 
legend=dict(font_size=14), 
barmode='group',
font_family='Open Sans',  
xaxis=dict(tickangle=-45, tickfont_size=16),
yaxis=dict(title='<b>Precipitação (mm)', titlefont_size=16, tickfont_size=16), 
annotations=[dict(text='<b>Clique nas legendas para <br>habilitar as séries históricas!', xref='paper', yref='paper', x=1.11, y=0.75, arrowhead=1, arrowwidth=2, arrowcolor='black', ax=0, ay=45, font=dict(color='black', size=14))]
)

fig2.show()
#fig2.write_html('C:\\Users\\daniel\\Documents\\Github\\geohmount\\geohmount-plotly\\sazonalidade-chuva\\sazonalidade.html', include_plotlyjs='cdn', config=config)